In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime
from keras.callbacks import History
history = History()
from jinja2 import Environment, FileSystemLoader
import json
import numpy as np
import os
import pandas as pd
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import math

Mounted at /content/drive


### Parameters to specify

In [ ]:
#example: Q16.8 -> precision = 8, wordsize = 24, double = 48
#specify:
PRECISION = 8
INTEGER = 8
WORDSIZE = INTEGER + PRECISION
D_WORDSIZE = 2*WORDSIZE
SLACK = 8-(WORDSIZE%8)


nn_topo_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-reports/nn_model_parameters_16-09-2022 14:40:08_with_10_attributes.json"
load_model_title = '/content/drive/MyDrive/network-traffic-classification-main/tf-models/model_with_10_attributes_12_hidden_nodes.csv'
path_test = f'/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_test_1_2000_samples.csv'

In [ ]:
n_bits = WORDSIZE
#for jinja template
wordsize = WORDSIZE
precision = PRECISION
slack = SLACK
print(slack)

8


### Load NN parameters and model

In [ ]:
#Load file with NN topology
with open(nn_topo_title, "r") as f:
  rep = json.load(f)

#Get NN topology and information to load correct dataframes
attributes_array = rep["attributes"]
number_of_attributes = rep["number_of_attributes"]
hidden_layer_nodes = rep["hidden_layer_nodes"]
output_layer_nodes = rep["output_layer_nodes"]
accuracy_test = rep["accuracy_test"]
batch_size = rep["batch_size"]
epochs = rep["epochs"]
number_of_samples = rep["number_of_samples"]
df_number = rep["df_number"]

#load dataframes and adequate them for file generation
#path_train = '/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_train_with%d'%number_of_samples + 'samples_per_class%d'%df_number + '.csv'
#df_train_raw = pd.read_csv(path_train)
df_test_raw = pd.read_csv(path_test)
#display(df_test_raw)
df_test = df_test_raw[attributes_array]
display(df_test)
target_test = df_test_raw['target']


#load model to get weights
print(load_model_title)
model = tf.keras.models.load_model(load_model_title)

norm_layer_weights = model.layers[0].get_weights()[0] #weight
norm_layer_biases  = model.layers[0].get_weights()[1] #bias
hidden_layer_weights = model.layers[1].get_weights()[0] #weight
hidden_layer_biases  = model.layers[1].get_weights()[1] #bias
out_layer_weights = model.layers[2].get_weights()[0] #weight
out_layer_biases  = model.layers[2].get_weights()[1] #bias
print('\nInput -> Normalization Layer bias:\n',norm_layer_biases)
print('\nInput -> Normalization Layer weights:\n',norm_layer_weights)
print('\nNormalization -> Hidden Layer bias:\n',hidden_layer_biases)
print('\nNormalization -> Hidden Layer weights:\n',hidden_layer_weights)
print('\nHidden -> Output Layer bias:\n',out_layer_biases)
# print('\nHidden -> Output Layer weights:\n',out_layer_weights,'\n')

,82,83,81,89,15,22,84,90,88,79
0,1368,189,189,55072,1434,1420,130,24624,24624,1380
1,536,64,536,64885,590,576,115,61440,61440,1432
2,1460,238,1460,64060,1514,1500,180,59195,61440,1460
3,142,375,375,16384,429,415,142,8192,61440,1460
4,141,558,558,5840,612,598,141,8192,61440,1460
...,...,...,...,...,...,...,...,...,...,...
23995,1380,48,176,15408,1434,1420,56,64959,65535,1380
23996,152,48,176,64140,230,216,56,64599,65535,1380
23997,152,48,176,64140,230,216,56,64599,65535,1380
23998,152,48,176,64140,230,216,56,64599,65535,1380


/content/drive/MyDrive/network-traffic-classification-main/tf-models/model_with_10_attributes_12_hidden_nodes.csv

Input -> Normalization Layer bias:
 [3.10864875e+05 3.83604805e+04 2.38350359e+05 5.56681216e+08
 2.82874312e+05 2.82960719e+05 4.40115586e+04 6.78492800e+08
 6.64743424e+08 1.08068016e+05]

Input -> Normalization Layer weights:
 [  481.05142   111.05538   442.2585  22201.584     801.3764    787.31824
    95.09093 34638.473   37587.91     1307.8844 ]

Normalization -> Hidden Layer bias:
 [ 1.5291731   0.51698875  3.0302162  -1.3996145   2.4409242   0.6488705
  0.6272663  -0.52198774  0.41071412 -0.18098997 -0.973893   -0.9532637 ]

Normalization -> Hidden Layer weights:
 [[ 1.87923267e-01  2.53490001e-01  1.71003222e+00  6.18668795e-01
  -2.53634787e+00 -6.45217776e-01 -2.97108674e+00 -8.84255528e-01
   3.76441985e-01  5.59191331e-02 -1.81314063e+00 -5.18236935e-01]
 [ 3.19220424e-01 -3.21883750e+00  2.16120386e+00 -2.91942477e+00
   9.28733349e-01 -3.40278912e+00  2.63627

In [ ]:
def num_to_c2(num, wordsize):
  if(num < 0):
    return 2**wordsize + num
  return num

### file generator for P4 csv input

In [ ]:
df_test.to_csv(f"/content/drive/MyDrive/network-traffic-classification-main/code/csv_files_input/df_test_{df_number}_with_{number_of_attributes}_attributes.csv", index=False)
target_test.to_csv(f"/content/drive/MyDrive/network-traffic-classification-main/csv-files/test_target_{df_number}_with_{number_of_attributes}_attributes.csv", index=False)

###  file generator for TF predictions
to be compared to P4_prediction in the analiser

In [ ]:
tf_predictions_probabilities = model.predict(df_test)
tf_loss, tf_acc =  model.evaluate(df_test,  target_test, verbose=2, batch_size=batch_size)

tf_predictions = []

for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)
  j_max = x.argmax()
  tf_predictions.append(j_max)

conf_m = tf.math.confusion_matrix(target_test,tf_predictions)
print(conf_m)

df_tf = pd.DataFrame(tf_predictions_probabilities)
df_tf.insert(loc = 0, column = "tf_predictions", value = tf_predictions)
#print(df_tf)

title_for_analiser = "/content/drive/MyDrive/network-traffic-classification-main/csv-files/tf_outputs_classifications.csv"
df_tf.to_csv(title_for_analiser, index=False)


750/750 [==============================] - 1s 1ms/step
2400/2400 - 3s - loss: 1.0367 - accuracy: 0.8036 - 3s/epoch - 1ms/step
tf.Tensor(
[[1607    2    1   22   28  281    0   38    1    9   11    0]
 [   0 1810   52    1    3   38   12   10   33   17   24    0]
 [   0   28 1876    0    0    9   42    3    7    0   35    0]
 [   0   26   82 1668   19   26    9    7   99   22   42    0]
 [ 381   12   20    0 1426   91    6   11   44    6    3    0]
 [ 263   25    9   12   22 1292   16   29  283    5   33   11]
 [   0    3    5    0    0    7 1980    0    0    0    5    0]
 [   4    5    6   90    9    5    7 1810   32   30    2    0]
 [   0   12    0  100    0   92   14   23 1692   12    7   48]
 [   0   17   17    0    4   19    0    8    0 1927    8    0]
 [   0    0  213    0    0   68    0    0    0    0 1719    0]
 [   0    0    0    0  519  516    0    0    0  486    0  479]], shape=(12, 12), dtype=int32)


### Topology file generator for sx

In [ ]:
weights = norm_layer_weights
biases = norm_layer_biases
#number of norm_layer_nodes = number_of_attributes
for i in range(number_of_attributes):
  sx_id = 1+i
  sx_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": 1
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": sx_id
        }
      },
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": sx_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": 1
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": 1
        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 5
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 2
        }
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : sx_id,
        "replicas": []
      }
    ]
  }
  for h in range(hidden_layer_nodes):
    sx_entries["multicast_group_entries"][0]["replicas"].append(
      {
        "egress_port" :51+h,
        "instance": 1
      }
    )



  #print(sx_entries["table_entries"])
  attr_name = attributes_array[i]

  for j in sorted(df_test[attr_name].unique()):
    if attr_name == "83":
      #print(j)
      if j == 0 or j == 1460:

        j = int(j)
        print(f"j:{j} ,weights[i]:{weights[i]}, math.sqrt(biases[i]:{math.sqrt(biases[i])}")

        normalized_value = (j - weights[i]) / math.sqrt(biases[i])
        print(f"normalized_value:{normalized_value}")

        fp_normalized_value = int(normalized_value * (2**PRECISION))
        print(f"fp_normalized_value:{fp_normalized_value}")

        c2_normalized_value = num_to_c2(fp_normalized_value, n_bits)
        print(f"c2_normalized_value:{c2_normalized_value}")

        sx_entries["table_entries"].append(
          {
            "table": "MyIngress.tab_normalization",
            "match": {
              "hdr.ann.data": j
            },
            "action_name": "MyIngress.set_normalization",
            "action_params": {
              "normalized_data": c2_normalized_value
            }
          }
        )


#   print(sx_entries["table_entries"])

#   sx_file_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/s%d"%sx_id + "-runtime.json"
#   with open(sx_file_title, "w") as f:
#     json.dump(sx_entries, f, indent = 2)

# print(json.dumps(sx_entries, indent = 2))

j:0 ,weights[i]:111.05538177490234, math.sqrt(biases[i]:195.85831733360214
normalized_value:-0.5670189721161731
fp_normalized_value:-145
c2_normalized_value:65391
j:1460 ,weights[i]:111.05538177490234, math.sqrt(biases[i]:195.85831733360214
normalized_value:6.887349164383268
fp_normalized_value:1763
c2_normalized_value:1763


### Topology file generator for s5x

In [ ]:
weights = hidden_layer_weights
biases = hidden_layer_biases
expected_stimuli = sum([1<<(i + 1) for i in range(number_of_attributes)])

for i in range(hidden_layer_nodes):
  s5x_id = 51+i
  s5x_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": s5x_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": number_of_attributes
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": expected_stimuli
        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 1
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 3
        }
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : s5x_id,
        "replicas":[]
      }
    ]
  }

  for h in range(output_layer_nodes):
    s5x_entries["multicast_group_entries"][0]["replicas"].append(
      {
        "egress_port" :101+h,
        "instance": 1
      }
    )

  fp_value = int(biases[i] * (2**PRECISION))
  c2_value = num_to_c2(fp_value, n_bits)
  s5x_entries["table_entries"].append(
    {
      "table": "MyIngress.tab_neuron_bias",
      "default_action": True,
      "action_name": "MyIngress.set_neuron_bias",
      "action_params": {
        "neuron_bias": c2_value
      }
    },
  )
  for j in range(number_of_attributes):
    fp_value = int(weights[j][i] * (2**PRECISION))
    c2_value = num_to_c2(fp_value, n_bits)
    s5x_entries["table_entries"].append(
      {
        "table": "MyIngress.tab_n2n_weight",
        "match": {
          "hdr.ann.neuron_id": j+1
        },
        "action_name": "MyIngress.set_n2n_weight",
        "action_params": {
          "n2n_weight":  c2_value
        }
      },
    )
    s5x_entries["table_entries"].append(
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": j+1
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": s5x_id
        }
      }
    )
  s5x_file_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/s%d"%s5x_id + "-runtime.json"
  with open(s5x_file_title, "w") as f:
    json.dump(s5x_entries, f, indent = 2)

print(json.dumps(s5x_entries, indent = 2))

{
  "target": "bmv2",
  "p4info": "build/ANN.p4.p4info.txt",
  "bmv2_json": "build/ANN.json",
  "table_entries": [
    {
      "table": "MyIngress.tab_neuron_id",
      "default_action": true,
      "action_name": "MyIngress.set_neuron_id",
      "action_params": {
        "neuron_id": 62
      }
    },
    {
      "table": "MyIngress.tab_n_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_n_expected_stimuli",
      "action_params": {
        "n_expected_stimuli": 10
      }
    },
    {
      "table": "MyIngress.tab_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_expected_stimuli",
      "action_params": {
        "expected_stimuli": 2046
      }
    },
    {
      "table": "MyIngress.tab_agg_func",
      "default_action": true,
      "action_name": "MyIngress.set_agg_func",
      "action_params": {
        "agg_func": 1
      }
    },
    {
      "table": "MyIngress.tab_activation_func",
      "default_action": t

In [ ]:
# #expected stimuli
# #hidden
# sum([1<<(i + 1) for i in range(n)])

# #output
# sum([1<<(i + 51) for i in range(n)])




# #expected stimuli
# import sys

# def main(neurons):
#     neurons = [int(x) for x in neurons.split(",")]
#     expected_stimuli = 0
#     for x in neurons:
#         expected_stimuli = expected_stimuli | 1 << x

#     print("dec:", expected_stimuli)
#     print("bin:", bin(expected_stimuli))

# if __name__ == '__main__':
#     main(sys.argv[1])

# #6
# #15762598695796736

### Topology file generator for s1xx

In [ ]:
weights = out_layer_weights
biases = out_layer_biases
expected_stimuli = sum([1<<(i + 51) for i in range(hidden_layer_nodes)])

for i in range(output_layer_nodes):
  s1xx_id = 101+i
  s1xx_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": s1xx_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": hidden_layer_nodes
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": expected_stimuli

        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 1
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 2
        }
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : s1xx_id,
        "replicas":[
          {
            "egress_port" :126,
            "instance": 1
          }
        ]
      }
    ]
  }

  fp_value = int(biases[i] * (2**PRECISION))
  c2_value = num_to_c2(fp_value, n_bits)
  s1xx_entries["table_entries"].append(
    {
      "table": "MyIngress.tab_neuron_bias",
      "default_action": True,
      "action_name": "MyIngress.set_neuron_bias",
      "action_params": {
        "neuron_bias": c2_value
      }
    },
  )

  for j in range(hidden_layer_nodes):
    fp_value = int(weights[j][i] * (2**PRECISION))
    c2_value = num_to_c2(fp_value, n_bits)
    s1xx_entries["table_entries"].append(
      {
        "table": "MyIngress.tab_n2n_weight",
        "match": {
          "hdr.ann.neuron_id": j+51
        },
        "action_name": "MyIngress.set_n2n_weight",
        "action_params": {
          "n2n_weight":  c2_value
        }
      }
    )
    s1xx_entries["table_entries"].append(
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": j+51
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": s1xx_id
        }
      }
    )

  #print(s1xx_entries["table_entries"])
  s1xx_file_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/s%d"%s1xx_id + "-runtime.json"
  with open(s1xx_file_title, "w") as f:
    json.dump(s1xx_entries, f, indent = 2)

print(json.dumps(s1xx_entries, indent = 2))

{
  "target": "bmv2",
  "p4info": "build/ANN.p4.p4info.txt",
  "bmv2_json": "build/ANN.json",
  "table_entries": [
    {
      "table": "MyIngress.tab_neuron_id",
      "default_action": true,
      "action_name": "MyIngress.set_neuron_id",
      "action_params": {
        "neuron_id": 112
      }
    },
    {
      "table": "MyIngress.tab_n_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_n_expected_stimuli",
      "action_params": {
        "n_expected_stimuli": 12
      }
    },
    {
      "table": "MyIngress.tab_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_expected_stimuli",
      "action_params": {
        "expected_stimuli": 9221120237041090560
      }
    },
    {
      "table": "MyIngress.tab_agg_func",
      "default_action": true,
      "action_name": "MyIngress.set_agg_func",
      "action_params": {
        "agg_func": 1
      }
    },
    {
      "table": "MyIngress.tab_activation_func",
      "de

### Topology file generator for topology.json


In [ ]:
#definition of topology
topology = {
    'hosts': {
        'h1': {'ip': '10.0.1.1/24', 'mac': '08:00:00:00:01:11',
                'commands':['route add default gw 10.0.1.10 dev eth0',
                            'arp -i eth0 -s 10.0.1.10 08:00:00:00:01:00']},
        'h2': {'ip': '10.0.2.2/24', 'mac': '08:00:00:00:02:22',
                'commands':['route add default gw 10.0.2.20 dev eth0',
                            'arp -i eth0 -s 10.0.2.20 08:00:00:00:02:00']}
    },
    'switches': {},
    'links': []
}

#populate switches dict
for i in range(number_of_attributes):
  switch_name = 's%d'%(i+1)
  switch_content = { 'runtime_json' : 'topology/s%d'%(i+1) +'-runtime.json' }
  topology['switches'][switch_name] = switch_content

for i in range(hidden_layer_nodes):
  switch_name = 's%d'%(i+51)
  switch_content = { 'runtime_json' : 'topology/s%d'%(i+51) +'-runtime.json' }
  topology['switches'][switch_name] = switch_content

for i in range(output_layer_nodes):
  switch_name = 's%d'%(i+101)
  switch_content = { 'runtime_json' : 'topology/s%d'%(i+101) +'-runtime.json' }
  topology['switches'][switch_name] = switch_content

topology['switches']['s126'] = { 'runtime_json' : 'topology/s126-runtime.json' }

#populate links list
for i in range(number_of_attributes):
  topology['links'].append(["h1", "s%d"%(i+1)+"-p1"])

for i in range(number_of_attributes):
  for j in range(hidden_layer_nodes):
    topology['links'].append(["s%d"%(i+1)+"-p%d"%(j+51), "s%d"%(j+51)+"-p%d"%(i+1)])

for i in range(hidden_layer_nodes):
  for j in range(output_layer_nodes):
    topology['links'].append(["s%d"%(i+51)+"-p%d"%(j+101), "s%d"%(j+101)+"-p%d"%(i+51)])

for i in range(output_layer_nodes):
  topology['links'].append(["s%d"%(i+101)+"-p126", "s126-p%d"%(i+101)])

topology['links'].append(["s126-p2", "h2"])

display(topology)

topology_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/topology.json"
with open(topology_title, "w") as f:
  json.dump(topology, f, indent = 2)


{'hosts': {'h1': {'ip': '10.0.1.1/24',
   'mac': '08:00:00:00:01:11',
   'commands': ['route add default gw 10.0.1.10 dev eth0',
    'arp -i eth0 -s 10.0.1.10 08:00:00:00:01:00']},
  'h2': {'ip': '10.0.2.2/24',
   'mac': '08:00:00:00:02:22',
   'commands': ['route add default gw 10.0.2.20 dev eth0',
    'arp -i eth0 -s 10.0.2.20 08:00:00:00:02:00']}},
 'switches': {'s1': {'runtime_json': 'topology/s1-runtime.json'},
  's2': {'runtime_json': 'topology/s2-runtime.json'},
  's3': {'runtime_json': 'topology/s3-runtime.json'},
  's4': {'runtime_json': 'topology/s4-runtime.json'},
  's5': {'runtime_json': 'topology/s5-runtime.json'},
  's6': {'runtime_json': 'topology/s6-runtime.json'},
  's7': {'runtime_json': 'topology/s7-runtime.json'},
  's8': {'runtime_json': 'topology/s8-runtime.json'},
  's9': {'runtime_json': 'topology/s9-runtime.json'},
  's10': {'runtime_json': 'topology/s10-runtime.json'},
  's51': {'runtime_json': 'topology/s51-runtime.json'},
  's52': {'runtime_json': 'topology/

### Jinja template to generate ann.p4, ann-header.py and ann_tester.py

In [ ]:
environment = Environment(loader=FileSystemLoader("/content/drive/MyDrive/network-traffic-classification-main/jinja/templates/"))
template = environment.get_template("ANN_tester_b.py")
content = template.render(
    wordsize= wordsize,
    precision= precision,
    integer= wordsize - precision,
    fraction = precision,
    slack = slack,
    df_number = df_number,
    number_of_attributes = number_of_attributes

)
filename = "/content/drive/MyDrive/network-traffic-classification-main/code/ANN_tester_b.py"
with open(filename, mode="w", encoding="utf-8") as message:
    message.write(content)
    print(f"... wrote {filename}")

###################################################
template = environment.get_template("ANN_hdrs.py")
content = template.render(
    wordsize= wordsize,
    slack = slack,
)
filename = "/content/drive/MyDrive/network-traffic-classification-main/code/ANN_hdrs.py"
with open(filename, mode="w", encoding="utf-8") as message:
    message.write(content)
    print(f"... wrote {filename}")

###################################################
template = environment.get_template("ANN.p4")
content = template.render(
    wordsize= wordsize,
    precision= precision,
    double = wordsize*2,
    slack = slack,
)
filename = "/content/drive/MyDrive/network-traffic-classification-main/code/ANN.p4"
with open(filename, mode="w", encoding="utf-8") as message:
    message.write(content)
    print(f"... wrote {filename}")

... wrote /content/drive/MyDrive/network-traffic-classification-main/code/ANN_tester_b.py
... wrote /content/drive/MyDrive/network-traffic-classification-main/code/ANN_hdrs.py
... wrote /content/drive/MyDrive/network-traffic-classification-main/code/ANN.p4
